# Image measurement: object morphology


### 1. Overview & learning objectives
In this notebook we will measure cells.  

With this notebook we will:

1. Learn about a useful function to measure both morphology and intensity features.

1. Measure the area and perimeter of the cells that we segmented.

1. Use plots to display the measurement results.

### 2. Measuring cell size
In our last lecture, we used the watershed algorithm to segment the cells. Run this cell to reproduce what we did so far:

In [ ]:
import matplotlib.pyplot as plt
import numpy
import scipy.ndimage as ndimage
from skimage import exposure, filters, feature, io, morphology
from skimage.segmentation import watershed

# Read image from disk.
animage = io.imread('cells.tif')

# Gaussian smoothing to facilitate edge detection.
animage_smooth = filters.gaussian(animage, sigma=2, preserve_range=True)

# Contrast stretch.
animage_rescaled = exposure.rescale_intensity(
    animage_smooth, out_range=numpy.uint8)

# Local threshold.
amask = animage_rescaled >= filters.threshold_local(
    animage_rescaled, 33, method='gaussian')

# Closing
amask_closed = morphology.binary_closing(amask, morphology.disk(3))

# Distance transform.
dt = ndimage.distance_transform_edt(numpy.invert(amask_closed))

# Seed identification.
coords_maxima = feature.peak_local_max(dt, labels=morphology.label(
    numpy.invert(amask_closed)), num_peaks_per_label=1, exclude_border=False)

# Create seed image.
seed_image = numpy.zeros(animage.shape)

# For each seed ...
for label, seed_xy in enumerate(coords_maxima):
    # ... set the value of the corresponding pixel to a different value.
    seed_image[seed_xy[0], seed_xy[1]] = label + 1

watershed_segm = watershed(animage_rescaled, markers=seed_image, watershed_line=True)

# Display image and seeds.
fig, axs = plt.subplots(1, 2)
axs[0].imshow(animage, cmap='Greys_r')
axs[1].imshow(watershed_segm, cmap='inferno')
plt.show()

Our last step is to measure the cells. The method **skimage.measure.regionprops** is an excellent tool to do this. Take a look at its documentation. Under the **Notes** section there is a list of features that regionprops measures, including area, perimeter, pixel value, etc.

We can use **regionprops** to measure all the cells in an image. The measurements can be accessed and collected into convenient lists like this:

In [ ]:
# Measure labeled image.
cell_props = regionprops(watershed_segm)  

# Define two empty lists to store areas and perimeters.
cell_areas = []
cell_perimeters = []

# Loop through the measurements for each cell ...
for acell in cell_props:
    # ... and store both area and perimeter in the corresponding lists.
    cell_areas.append(acell.area)
    cell_perimeters.append(acell.perimeter)

# Print the area and perimeter measurements.
print(cell_areas)
print(cell_perimeters)

### 3. Data visualization

Plotting measurement results is often more useful that printing each one of the measurements. We will use **matplotlib** to generate plots. Below is an example from the [matplotlib website](https://matplotlib.org/2.0.0/examples/statistics/boxplot_vs_violin_demo.html) that generates a violin and a box plot for random, test data:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))

# generate some random test data
all_data = [numpy.random.normal(0, std, 100) for std in range(6, 10)]

# plot violin plot
axes[0].violinplot(all_data,
                   showmeans=False,
                   showmedians=True)
axes[0].set_title('violin plot')

# plot box plot
axes[1].boxplot(all_data)
axes[1].set_title('box plot')

# adding horizontal grid lines
for ax in axes:
    ax.yaxis.grid(True)
    ax.set_xticks([y+1 for y in range(len(all_data))])
    ax.set_xlabel('xlabel')
    ax.set_ylabel('ylabel')

# add x-tick labels
plt.setp(axes, xticks=[y+1 for y in range(len(all_data))],
         xticklabels=['x1', 'x2', 'x3', 'x4'])
plt.show()

Use the code above as a guide to generate a figure with two plots, each of them comparing area and perimeter for the cells in the original image, displayed as a violin plot on the left, and as a box plot on the right.